## Get multi-organism model (using ConsortiumModels)

In [2]:
# Change directories to make "concerto" module visible
import os 
curr_dir = os.getcwd()
if not curr_dir.endswith("CONCERTO"):
    os.chdir(os.path.dirname(curr_dir))

import concerto


In [3]:
# Load community models. May take a few minutes..
from binary_models.load_ternary import load_community_model as load_ternary_model
from binary_models.flux_visualizations import create_explicit_carbon
from binary_models.load_av_rt import load_community_model as load_av_rt_model

ternary_community, model_file = load_ternary_model()
ternary_community = create_explicit_carbon(ternary_community)

av_rt_community, model_file = load_av_rt_model()


[15:52:34] WARNING  Could not identify an external          boundary_types.py:90
                    compartment by name and choosing one                        
                    with the most boundary reactions. That                      
                    might be complete nonsense or change                        
                    suddenly. Consider renaming your                            
                    compartments using `Model.compartments`                     
                    to fix this.                                                
Running ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:08
Running                                            0% -:--:--[15:53:06] WARNING  The table includes a `file` column even     community.py:189
                    though a model database is used. Will                       
                    ignore it and use the model database                        
                    instead.                                       

In [4]:
ternary_community

Name,One
Memory address,7fefc31c93c0
Number of metabolites,5494
Number of reactions,6475
Number of genes,3337
Number of groups,0
Objective expression,1.0*community_objective
Compartments,"c__Rhodosporidium, m__Rhodosporidium, r__Rhodosporidium, e__Rhodosporidium, x__Rhodosporidium, v__Rhodosporidium, n__Rhodosporidium, g__Rhodosporidium, d__Rhodosporidium, m, e__Synechococcus, c__Synechococcus, cx__Synechococcus, u__Synechococcus, p__Synechococcus, um__Synechococcus, cm__Synechococcus, cytosol__Synechococcus, c__Azotobacter, e__Azotobacter, p__Azotobacter"


In [5]:
av_rt_community

Name,One
Memory address,7fefac5d8070
Number of metabolites,4551
Number of reactions,5366
Number of genes,2415
Number of groups,0
Objective expression,1.0*community_objective
Compartments,"c__Azotobacter, e__Azotobacter, p__Azotobacter, m, c__Rhodosporidium, m__Rhodosporidium, r__Rhodosporidium, e__Rhodosporidium, x__Rhodosporidium, v__Rhodosporidium, n__Rhodosporidium, g__Rhodosporidium, d__Rhodosporidium"


## Optimize Model $\rightarrow$ Get Flux dataframe

In [6]:
ternary_result = ternary_community.optimize(fluxes=True, pfba=False)
ternary_result

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.333333,115.422433,2493,2024
Rhodosporidium,0.333333,36.594531,2424,2076
Synechococcus,0.333333,2.021514,1091,927
medium,NaN,NaN,467,467


In [7]:
ternary_result.fluxes

reaction,10FTHFGLULL,10FTHFGLULLm,12DGR120tipp,12DGR140tipp,12DGR141tipp,12DGR160tipp,12DGR161tipp,12DGR180tipp,12DGR181tipp,12PPDRte,12PPDRtex,12PPDRtpp,12PPDSt,12PPDStex,12PPDStpp,12PPDt,13BGHe,13DAMPPOX,13GS,13PPDH,14GLUCANabcpp,14GLUCANtexi,16GS,23CAMPtex,23CCMPtex,23CGMPtex,23CUMPtex,23DAPPAt2pp,23DAPPAtex,23DK5MPPISO,23DOGULNt4pp,26DAHtex,2AGPA120tipp,2AGPA140tipp,2AGPA141tipp,2AGPA160tipp,2AGPA161tipp,2AGPA180tipp,2AGPA181tipp,2AGPE120tipp,...,XPK,XPPT,XTSNH,XTSNt2rpp,XTSNtex,XTSNtr,XYLK,XYLR,XYLTD_D,XYLTpp,XYLTt,XYLTtex,XYLUDte,XYLUR,XYLUR2,XYLUt2pp,XYLUtex,XYLabcpp,XYLt,XYLt2pp,XYLtex,YUMPS,ZCARDS,ZCAROTDH1,ZCAROTDH2,ZDS,ZISO,ZN2t3pp,ZN2t4pp,ZNabcpp,ZXANHX,ZYMSTATer_RT,ZYMSTESTH_RT,ZYMSTESTtrd,ZYMSTt,ZYMSTtr,Zn2t,Zn2tex,ilvg,r0647
compartment,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Azotobacter,0.000094,NaN,-0.000014,-0.000014,-0.000016,-0.000064,-0.000092,-0.000068,-0.000011,NaN,-0.000856,-0.000854,NaN,-33.312147,-33.312144,-66.62429,NaN,NaN,NaN,-0.000002,-4.507686e-07,-4.495188e-07,NaN,0.0,0.0,0.0,0.0,-7.642507e-07,-7.621702e-07,0.000109,-0.000002,-0.085747,-0.00003,-0.000031,-0.000039,-0.000024,0.0,-0.00003,0.0,0.000018,...,NaN,0.00317,NaN,-0.022635,-0.022633,NaN,200.392603,NaN,NaN,100.181694,NaN,100.181717,NaN,NaN,100.181672,-7.642507e-07,-7.621702e-07,0.000156,NaN,-0.000157,-3.730982e-07,NaN,NaN,NaN,NaN,NaN,NaN,-0.000202,0.002947,0.042051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039306,0.000044,NaN
Rhodosporidium,NaN,0.000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01548,NaN,NaN,-0.024631,NaN,NaN,NaN,-0.048061,-0.000012,35.039368,NaN,NaN,NaN,6.488799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1812.973565,-0.02061,-0.014099,NaN,NaN,99.901204,91.239997,99.826474,197.367945,NaN,-101.204217,NaN,0.720872,198.745796,NaN,NaN,NaN,NaN,99.826562,NaN,NaN,0.000037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.049237,0.009792,-0.000491,0.011578,0.021366,0.018663,NaN,NaN,-0.000008
Synechococcus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001525,-0.000541,-0.000247,-0.000702,0.001227,NaN,NaN,0.000004,-0.007104,NaN,NaN,NaN,NaN,NaN,NaN,0.000004,NaN,NaN
medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
av_rt_result = av_rt_community.optimize(fluxes=True, pfba=False)
av_rt_result

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.5,41.289363,2492,2024
Rhodosporidium,0.5,2.293007,2423,2076
medium,NaN,NaN,451,451


In [10]:
av_rt_result.fluxes

reaction,10FTHFGLULL,10FTHFGLULLm,12DGR120tipp,12DGR140tipp,12DGR141tipp,12DGR160tipp,12DGR161tipp,12DGR180tipp,12DGR181tipp,12PPDRte,12PPDRtex,12PPDRtpp,12PPDSt,12PPDStex,12PPDStpp,12PPDt,13BGHe,13DAMPPOX,13GS,13PPDH,14GLUCANabcpp,14GLUCANtexi,16GS,23CAMPtex,23CCMPtex,23CGMPtex,23CUMPtex,23DAPPAt2pp,23DAPPAtex,23DK5MPPISO,23DOGULNt4pp,26DAHtex,2AGPA120tipp,2AGPA140tipp,2AGPA141tipp,2AGPA160tipp,2AGPA161tipp,2AGPA180tipp,2AGPA181tipp,2AGPE120tipp,...,XANt,XANt2pp,XANtex,XAO2x,XAOx,XMPtex,XPK,XPPT,XTSNH,XTSNt2rpp,XTSNtex,XTSNtr,XYLK,XYLR,XYLTD_D,XYLTpp,XYLTt,XYLTtex,XYLUDte,XYLUR,XYLUR2,XYLUt2pp,XYLUtex,XYLabcpp,XYLt,XYLt2pp,XYLtex,YUMPS,ZN2t3pp,ZN2t4pp,ZNabcpp,ZYMSTATer_RT,ZYMSTESTH_RT,ZYMSTESTtrd,ZYMSTt,ZYMSTtr,Zn2t,Zn2tex,ilvg,r0647
compartment,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Azotobacter,0.00003,NaN,0.000009,0.00001,0.000009,0.000092,0.000082,0.000086,0.000015,NaN,0.004666,0.004662,NaN,-33.321188,-33.321193,-66.642382,NaN,NaN,NaN,-0.00001,-2.067587e-07,-2.072444e-07,NaN,0.0,0.0,0.0,0.0,4.894217e-08,4.807548e-08,0.000044,1.306446e-07,0.014968,0.00004,0.000044,0.000042,0.000055,0.0,0.000054,0.0,-0.000024,...,NaN,0.090638,0.090632,NaN,NaN,0.0,NaN,0.120833,NaN,0.001796,0.001788,NaN,0.006620,NaN,NaN,-0.534944,NaN,-0.534955,NaN,NaN,-0.534932,4.894217e-08,4.807548e-08,-0.000129,NaN,0.000128,-0.000003,NaN,-0.000111,0.00059,0.014564,NaN,NaN,NaN,NaN,NaN,NaN,0.014085,-0.000018,NaN
Rhodosporidium,NaN,-0.000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008388,NaN,NaN,-0.003211,NaN,NaN,NaN,0.007485,0.000002,2.195551,NaN,NaN,NaN,0.406577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.069294,NaN,NaN,1.267440e-07,-0.002718,NaN,-0.006226,-0.012556,-0.002129,NaN,NaN,0.029367,0.106717,0.00566,0.645588,NaN,0.579647,NaN,-0.02432,0.06027,NaN,NaN,NaN,NaN,0.005658,NaN,NaN,-0.000023,NaN,NaN,NaN,-0.000321,-0.000815,-0.000003,0.014397,0.013586,0.001169,NaN,NaN,-0.000003
medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Begin Visualization for Ternary Model

In [12]:
from concerto.utils import flux_visualization


### Organize subclasses for metabolites

Identify Synechococcus as the carbon carrier, Azotobacter as the nitrogen carrier, and Rhodosporidium as the jet_fuel producer.

In [13]:
ternary_org2group_dict = {
    'Synechococcus': 'C',
    'Azotobacter': 'N',
    'Rhodosporidium': 'jet_fuel'
}

In [14]:
### Note to Andrew: separate test to ensure external metabolites from the model are included in the flux_df yield a list of metabolites that were excluded

### Create, display and save Visualization object

In [16]:
ternary_viz_object, node_df, links_df = flux_visualization.create_fluxflow_viz(consort_model = ternary_community, 
                                           flux_df = ternary_result.fluxes, 
                                           org2group_dict = ternary_org2group_dict,
                                           flux_threshold = 90 # optional parameter; default = 0.01
                                           )

import pandas as pd

pd.set_option('display.max_rows', 500)
display(node_df.loc[-node_df['Xpos'].isna() & (node_df['Type']=='shared_m')].sort_values(by="Ypos"))


links_df.sort_values(by="Values")

In [17]:
# Display diagram in notebook
ternary_viz_object.show()

In [18]:
# Save diagram as an interactive HTML to file 
ternary_viz_object.update_layout(width=1600, height=900, margin={'t':10,'b':10})
ternary_viz_fname = "Alluvial_Diagram_ConsortiumModel_Ternary.html"
ternary_viz_object.write_html(ternary_viz_fname)

## Begin Visualization for Binary Model (AV_RT)

In [19]:
av_rt_org2group_dict = {
    'Azotobacter': 'N',
    'Rhodosporidium': 'jet_fuel'
}

In [20]:
av_rt_viz_object, node_df, links_df = flux_visualization.create_fluxflow_viz(consort_model = av_rt_community, 
                                                                             flux_df = av_rt_result.fluxes, 
                                                                             org2group_dict = av_rt_org2group_dict,
                                                                             flux_threshold = 90 # optional parameter; default = 0.01
                                                                             )

In [21]:
# Display diagram in notebook
av_rt_viz_object.show()

In [22]:
# Save diagram as an interactive HTML to file 
av_rt_viz_object.update_layout(width=1600, height=900, margin={'t':10,'b':10})
av_rt_viz_fname = "Alluvial_Diagram_ConsortiumModel_Binary_AV_RT.html"
av_rt_viz_object.write_html(av_rt_viz_fname)